In [1]:
import os
from collections import defaultdict
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
root_dir = "../../"

In [3]:
f = "net_202001_weekdayb.csv"
df_res1 = pd.read_csv(os.path.join(root_dir, "mapequation", f), index_col=0)
df_res1.head()

,clusters,flow,node,ori_index,level_1,level_2
ind,,,,,,
0,1:1,0.023884,TAMPINES EAST,40,1,1_1
1,1:2,0.015615,SERANGOON CENTRAL,69,1,1_2
2,1:3,0.015583,BEDOK NORTH,16,1,1_3
3,1:4,0.015119,GEYLANG EAST,10,1,1_4
4,1:5,0.014226,TOA PAYOH CENTRAL,68,1,1_5


In [4]:
f = "net_202001_weekendb.csv"
df_res2 = pd.read_csv(os.path.join(root_dir, "mapequation", f), index_col=0)
df_res2.head()

,clusters,flow,node,ori_index,level_1,level_2
ind,,,,,,
0,1:1,0.023627,TAMPINES EAST,40,1,1_1
1,1:2,0.019097,BOULEVARD,79,1,1_2
2,1:3,0.017946,GEYLANG EAST,10,1,1_3
3,1:4,0.016579,BEDOK NORTH,16,1,1_4
4,1:5,0.016276,SERANGOON CENTRAL,69,1,1_5


In [5]:
df_merge = pd.merge(df_res1[["node", "level_1"]], df_res2[["node", "level_1"]], left_on="node", right_on="node", suffixes=["_weekday", "_weekend"])
df_merge.head()

,node,level_1_weekday,level_1_weekend
0,TAMPINES EAST,1,1
1,SERANGOON CENTRAL,1,1
2,BEDOK NORTH,1,1
3,GEYLANG EAST,1,1
4,TOA PAYOH CENTRAL,1,7


In [6]:
df_merge.to_csv("results/mapequation_com.csv", index_label="ind")

In [7]:
data_dir = "../../data"

In [8]:
day = "weekday" # "weekend"
fp = os.path.join(data_dir, "net_202001_{}b.net".format(day))
dg1 = nx.read_pajek(fp)
dg1 = nx.DiGraph(dg1)
dg1.number_of_nodes(), dg1.number_of_edges()

(303, 30043)

In [9]:
day = "weekend"
fp2 = os.path.join(data_dir, "net_202001_{}b.net".format(day))
dg2 = nx.read_pajek(fp2)
dg2 = nx.DiGraph(dg2)
dg2.number_of_nodes(), dg2.number_of_edges()

(303, 30043)

In [10]:
def conn_entropy(dg, df, col, direction="out", edge_weight="weight"):
    node_dic = {}
    for n, v in zip(df["node"].tolist(), df[col].tolist()):
        node_dic[n] = v
    entropy_dic = {}
    number_of_all_community = len(list(set(df[col].tolist())))
    for n in dg.nodes():
        nb = None
        if direction=="out":
            nb = dg.out_edges(nbunch=n,data=True)
            nb = [ (v,d[edge_weight]) for u,v,d in nb ]
        else:
            nb = dg.in_edges(nbunch=n, data=True)
            nb = [ (u,d[edge_weight]) for u,v,d in nb ]
        if len(nb)<=1:
            entropy_dic[n] = -1
            continue
        tot_weight = sum([ w for c,w in nb ])
        partial = [ float(w)/tot_weight for c,w in nb ]
        cats = [ node_dic[c] for c,w in nb ]
        cat_set = list(set(cats))
        if len(cat_set)<=1:
            entropy_dic[n] = 0.
            continue
        proportions = { cat:0. for cat in cat_set }
        for cat,prop in zip(cats, partial):
            proportions[cat]+=prop
        #ent = -sum([ p*np.log(p) for p in proportions.values() ])/np.log(len(cat_set))
        ent = -sum([ p*np.log(p) for p in proportions.values() ])/np.log(number_of_all_community)
        #print(ent)
        entropy_dic[n] = ent
        #break
    return entropy_dic

In [11]:
df_com = pd.read_csv("results/mapequation_com.csv", index_col=0)
df_com.head()

,node,level_1_weekday,level_1_weekend
ind,,,
0,TAMPINES EAST,1,1
1,SERANGOON CENTRAL,1,1
2,BEDOK NORTH,1,1
3,GEYLANG EAST,1,1
4,TOA PAYOH CENTRAL,1,7


In [12]:
com_entropy_1_in = conn_entropy(dg1, df_com, "level_1_weekday", direction="in", edge_weight="weight")
com_entropy_1_out = conn_entropy(dg1, df_com, "level_1_weekday", direction="out", edge_weight="weight")
com_entropy_2_in = conn_entropy(dg2, df_com, "level_1_weekend", direction="in", edge_weight="weight")
com_entropy_2_out = conn_entropy(dg2, df_com, "level_1_weekend", direction="out", edge_weight="weight")
#check#["THE WHARVES"]

In [13]:
df_entropy = pd.DataFrame.from_dict({"weekday_com_ent_in":com_entropy_1_in, 
                                     "weekday_com_ent_out":com_entropy_1_out, 
                                     "weekend_com_ent_in":com_entropy_2_in, 
                                     "weekend_com_ent_out":com_entropy_2_out, 
                                     })
df_entropy.head()

,weekday_com_ent_in,weekday_com_ent_out,weekend_com_ent_in,weekend_com_ent_out
ADMIRALTY,0.285268,0.405529,0.355581,0.166908
AIRPORT ROAD,0.000000,0.000000,0.205652,0.196236
ALEXANDRA HILL,0.127008,0.116584,0.177661,0.163602
ALEXANDRA NORTH,0.077890,0.000000,0.123208,0.017938
ALJUNIED,0.144630,0.148059,0.225397,0.259171


In [14]:
com_res = pd.merge(df_com, df_entropy, left_on="node", right_index=True, suffixes=["_com", "entropy"])

In [15]:
com_res.head()

,node,level_1_weekday,level_1_weekend,weekday_com_ent_in,weekday_com_ent_out,weekend_com_ent_in,weekend_com_ent_out
ind,,,,,,,
0,TAMPINES EAST,1,1,0.145412,0.139498,0.184632,0.180285
1,SERANGOON CENTRAL,1,1,0.128915,0.129463,0.308747,0.314461
2,BEDOK NORTH,1,1,0.111336,0.115798,0.168210,0.155813
3,GEYLANG EAST,1,1,0.203366,0.200707,0.295818,0.283110
4,TOA PAYOH CENTRAL,1,7,0.291065,0.241715,0.612989,0.599574


In [16]:
com_res.to_csv("results/mapequation_com_entropy.csv", index_label="ind")